# KQL attack-hunting query catalog (Microsoft Sentinel + Defender XDR)

This notebook helps you **collect and organize KQL hunting queries by attack type** (aligned to **MITRE ATT&CK tactics**) and gives you a **starter pack of ready-to-run queries** for common detections.

It supports two major public catalogs:
- **Azure/Azure-Sentinel** (Microsoft Sentinel hunting queries + analytics rules)
- **microsoft/Microsoft-365-Defender-Hunting-Queries** (Defender XDR Advanced Hunting community queries)

> Notes  
> - KQL tables differ by product and connector. Some queries in these catalogs are specific to certain tables (e.g., `DeviceProcessEvents` in Defender XDR vs `SecurityEvent` / `SigninLogs` in Sentinel/Log Analytics).  
> - The downloader below pulls the *latest* versions from GitHub when you run it.  
> - You can export a combined index (CSV/JSON) and quickly search/query-text across all files.


## Sources (public query catalogs & docs)

- Azure Sentinel / Microsoft Sentinel unified repo (**Azure/Azure-Sentinel**)  
- Microsoft Defender XDR community hunting queries (**microsoft/Microsoft-365-Defender-Hunting-Queries**)  
- Kusto Query Language docs (Microsoft Learn)


In [ ]:
from __future__ import annotations

import os
import re
import io
import json
import zipfile
import shutil
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import pandas as pd

# --- Configuration ---
WORKDIR = Path("kql_catalog_workdir").resolve()
WORKDIR.mkdir(parents=True, exist_ok=True)

REPOS = [
    {
        "name": "Azure-Sentinel",
        "zip_url": "https://github.com/Azure/Azure-Sentinel/archive/refs/heads/master.zip",
        "subdirs_hint": ["Hunting Queries", "Detections", "Solutions"],
    },
    {
        "name": "Microsoft-365-Defender-Hunting-Queries",
        "zip_url": "https://github.com/microsoft/Microsoft-365-Defender-Hunting-Queries/archive/refs/heads/main.zip",
        "subdirs_hint": ["Queries", "Hunting Queries"],
    },
]

# ATT&CK tactics (common set) and simple path keyword mapping
TACTIC_KEYWORDS = {
    "reconnaissance": ["recon", "reconnaissance"],
    "resource development": ["resource", "dev"],
    "initial access": ["initialaccess", "initial access"],
    "execution": ["execution"],
    "persistence": ["persistence"],
    "privilege escalation": ["privilegeescalation", "privilege escalation"],
    "defense evasion": ["defenseevasion", "defense evasion", "evas"],
    "credential access": ["credentialaccess", "credential access", "creds"],
    "discovery": ["discovery"],
    "lateral movement": ["lateralmovement", "lateral movement"],
    "collection": ["collection"],
    "command and control": ["commandandcontrol", "command and control", "c2"],
    "exfiltration": ["exfiltration"],
    "impact": ["impact", "ransom", "destruction"],
}

# File extensions to include as "queries"
QUERY_EXTS = {".kql", ".csl", ".txt"}

print("Workdir:", WORKDIR)


In [ ]:
import requests

def download_zip(url: str) -> bytes:
    r = requests.get(url, timeout=120)
    r.raise_for_status()
    return r.content

def extract_zip_bytes(zip_bytes: bytes, dest: Path) -> Path:
    dest.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_bytes)) as zf:
        zf.extractall(dest)
    # Usually a single top-level folder exists inside the zip
    top = next(dest.iterdir())
    return top

def fetch_repo(repo: Dict, base: Path) -> Path:
    name = repo["name"]
    repo_dir = base / name
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    repo_dir.mkdir(parents=True, exist_ok=True)

    print(f"Downloading {name}...")
    zbytes = download_zip(repo["zip_url"])
    top = extract_zip_bytes(zbytes, repo_dir)
    print(f"Extracted to: {top}")
    return top

repo_roots = {}
for repo in REPOS:
    repo_roots[repo["name"]] = fetch_repo(repo, WORKDIR)

repo_roots


In [ ]:
def normalize_path(p: Path) -> str:
    # Normalize to lowercase path with '/' separators for matching
    return str(p).replace('\\', '/').lower()

def infer_tactic_from_path(p: Path) -> Optional[str]:
    path_l = normalize_path(p)
    compact = path_l.replace("_", "").replace("-", "").replace(" ", "")
    for tactic, kws in TACTIC_KEYWORDS.items():
        for kw in kws:
            kwc = kw.replace(" ", "")
            if kwc in compact:
                return tactic
    return None

TITLE_RE = re.compile(r"""(?im)^\s*(//|#)?\s*(title\s*:\s*|\[?title\]?\s*[:=-]\s*)?(?P<title>[^\n]{5,120})\s*$""")

def extract_title(text: str) -> Optional[str]:
    # Heuristics: first non-empty comment line, or YAML-like "title:"
    lines = [ln.strip() for ln in text.splitlines()]
    for ln in lines[:30]:
        if not ln:
            continue
        if ln.lower().startswith(("let ", "datatable", "union ", "search ", "where ", "summarize ")):
            break
        if ln.startswith("//") or ln.startswith("#") or "title:" in ln.lower():
            m = TITLE_RE.match(ln)
            if m:
                t = m.group("title").strip(" -#/\t")
                if 5 <= len(t) <= 120:
                    return t
    return None

def list_query_files(root: Path) -> List[Path]:
    files = []
    for ext in QUERY_EXTS:
        files.extend(root.rglob(f"*{ext}"))
    files = [p for p in files if p.is_file() and p.stat().st_size < 2_000_000]
    return sorted(files)

rows = []
for repo_name, root in repo_roots.items():
    qfiles = list_query_files(root)
    for f in qfiles:
        try:
            text = f.read_text(errors="ignore")
        except Exception:
            continue
        title = extract_title(text) or f.stem
        tactic = infer_tactic_from_path(f) or "unclassified"
        rows.append({
            "repo": repo_name,
            "tactic": tactic,
            "title": title,
            "path": str(f.relative_to(root)),
            "ext": f.suffix.lower(),
            "bytes": f.stat().st_size,
        })

df = pd.DataFrame(rows).sort_values(["repo", "tactic", "title"]).reset_index(drop=True)
df.head(), df.shape


In [ ]:
# Summary: how many query files per repo / tactic?
summary = (
    df.groupby(["repo", "tactic"])
      .size()
      .reset_index(name="count")
      .sort_values(["repo", "count"], ascending=[True, False])
)
summary


In [ ]:
# Simple search helpers
def search_titles(keyword: str, repo: Optional[str]=None, tactic: Optional[str]=None, n: int=25) -> pd.DataFrame:
    d = df
    if repo:
        d = d[d["repo"].str.lower() == repo.lower()]
    if tactic:
        d = d[d["tactic"].str.lower() == tactic.lower()]
    kw = keyword.lower().strip()
    out = d[d["title"].str.lower().str.contains(kw, na=False)]
    return out.head(n)

def load_query(repo: str, rel_path: str) -> str:
    root = repo_roots[repo]
    p = root / rel_path
    return p.read_text(errors="ignore")

search_titles("ransom")  # example


In [ ]:
# Export the combined index for use elsewhere
OUTDIR = WORKDIR / "exports"
OUTDIR.mkdir(parents=True, exist_ok=True)

csv_path = OUTDIR / "kql_query_index.csv"
json_path = OUTDIR / "kql_query_index.json"

df.to_csv(csv_path, index=False)
df.to_json(json_path, orient="records", indent=2)

csv_path, json_path


## Starter pack: ready-to-run KQL queries by attack type

The following sections include **generic, parameterized queries** you can paste into:

- **Microsoft Defender XDR Advanced Hunting** (tables like `DeviceProcessEvents`, `DeviceNetworkEvents`, `EmailEvents`, `IdentityLogonEvents`, etc.)  
- **Microsoft Sentinel / Log Analytics** (tables like `SecurityEvent`, `SigninLogs`, `AuditLogs`, `AzureActivity`, `CommonSecurityLog`, etc.)

You may need to adapt table names depending on your environment.

Each section includes multiple patterns that commonly map to that ATT&CK tactic.


### Initial Access

Queries focused on suspicious logons, password spray, and abnormal access patterns.

**Suspicious sign-in bursts (Sentinel: SigninLogs)**

In [ ]:
// Sentinel (Azure AD / Entra ID) sign-in anomalies: multiple failures from same IP/user
let lookback = 24h;
SigninLogs
| where TimeGenerated >= ago(lookback)
| summarize failures=countif(ResultType != 0), successes=countif(ResultType == 0),
            users=dcount(UserPrincipalName), apps=dcount(AppDisplayName)
          by IPAddress, bin(TimeGenerated, 1h)
| where failures >= 20 and users >= 5
| order by failures desc

**Password spray style pattern (Defender XDR: IdentityLogonEvents)**

In [ ]:
// Defender XDR: many accounts with failures from same IP in a short window
let lookback = 24h;
let window = 15m;
IdentityLogonEvents
| where Timestamp >= ago(lookback)
| where ActionType in ("LogonFailed", "LogonFailure")
| summarize failures=count(), users=dcount(AccountUpn), sampleUsers=make_set(AccountUpn, 10)
          by IPAddress, bin(Timestamp, window)
| where failures >= 25 and users >= 10
| order by failures desc

**Rare country/geo sign-ins for user (Sentinel: SigninLogs)**

In [ ]:
// Detect user sign-ins from a new country compared to prior baseline
let baseline = 14d;
let recent = 1d;
let base =
    SigninLogs
    | where TimeGenerated between (ago(baseline) .. ago(recent))
    | summarize baseCountries=make_set(LocationDetails.countryOrRegion, 50) by UserPrincipalName;
SigninLogs
| where TimeGenerated >= ago(recent)
| summarize recentCountries=make_set(LocationDetails.countryOrRegion, 50) by UserPrincipalName
| join kind=leftouter base on UserPrincipalName
| extend newCountries=set_difference(recentCountries, baseCountries)
| where array_length(newCountries) > 0
| project UserPrincipalName, newCountries, recentCountries

### Execution

Look for suspicious process creation, encoded commands, LOLBins, and script interpreters.

**Encoded/obfuscated PowerShell (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback = 7d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where FileName =~ "powershell.exe" or FileName =~ "pwsh.exe"
| where ProcessCommandLine has_any ("-enc", "-encodedcommand", "FromBase64String", "IEX", "Invoke-Expression")
| project Timestamp, DeviceName, AccountName, FileName, ProcessCommandLine, InitiatingProcessFileName, InitiatingProcessCommandLine
| order by Timestamp desc

**Suspicious script engines (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=7d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where FileName in~ ("wscript.exe","cscript.exe","mshta.exe","rundll32.exe","regsvr32.exe")
| summarize count() by FileName, InitiatingProcessFileName, bin(Timestamp, 1d)
| order by count_ desc

**Process creation for common LOLBins (Sentinel: SecurityEvent 4688)**

In [ ]:
// Requires Windows Security Events ingested; EventID 4688 = process creation
let lookback = 7d;
let lolbins = dynamic(["rundll32.exe","regsvr32.exe","mshta.exe","certutil.exe","bitsadmin.exe","installutil.exe","wmic.exe"]);
SecurityEvent
| where TimeGenerated >= ago(lookback) and EventID == 4688
| where NewProcessName has_any (lolbins)
| project TimeGenerated, Computer, Account, NewProcessName, CommandLine, ParentProcessName
| order by TimeGenerated desc

### Persistence

Common persistence mechanisms: scheduled tasks, services, Run keys, startup folders.

**New scheduled task created (Sentinel: SecurityEvent 4698)**

In [ ]:
let lookback=14d;
SecurityEvent
| where TimeGenerated >= ago(lookback) and EventID == 4698
| project TimeGenerated, Computer, Account, TaskName=tostring(EventData.TaskName), Command=tostring(EventData.Command)
| order by TimeGenerated desc

**Service installed/created (Sentinel: SecurityEvent 7045 or 4697)**

In [ ]:
let lookback=14d;
SecurityEvent
| where TimeGenerated >= ago(lookback) and EventID in (4697, 7045)
| extend ServiceName=tostring(EventData.ServiceName), ImagePath=tostring(EventData.ImagePath)
| project TimeGenerated, Computer, Account, EventID, ServiceName, ImagePath
| order by TimeGenerated desc

**Run key modifications (Defender XDR: DeviceRegistryEvents)**

In [ ]:
let lookback=14d;
let runKeys = dynamic([
  @"HKEY_LOCAL_MACHINE\Software\Microsoft\Windows\CurrentVersion\Run",
  @"HKEY_CURRENT_USER\Software\Microsoft\Windows\CurrentVersion\Run"
]);
DeviceRegistryEvents
| where Timestamp >= ago(lookback)
| where RegistryKey has_any (runKeys)
| project Timestamp, DeviceName, ActionType, RegistryKey, RegistryValueName, RegistryValueData, InitiatingProcessFileName, InitiatingProcessCommandLine
| order by Timestamp desc

### Privilege Escalation

Admin group changes, suspicious token actions, UAC-bypass patterns, elevation events.

**Local admin group membership changes (Sentinel: SecurityEvent 4732/4728)**

In [ ]:
let lookback=14d;
SecurityEvent
| where TimeGenerated >= ago(lookback) and EventID in (4728, 4732) // added to global/local group
| extend GroupName=tostring(EventData.TargetUserName), Member=tostring(EventData.MemberName)
| project TimeGenerated, Computer, Account, EventID, GroupName, Member
| order by TimeGenerated desc

**Potential UAC bypass via fodhelper/CMSTP (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=14d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where FileName in~ ("fodhelper.exe","cmstp.exe","sdclt.exe")
| project Timestamp, DeviceName, AccountName, FileName, ProcessCommandLine, InitiatingProcessFileName, InitiatingProcessCommandLine
| order by Timestamp desc

**Process runs with high integrity (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=7d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where ProcessIntegrityLevel in ("High","System")
| summarize count(), sampleCmd=any(ProcessCommandLine) by FileName, AccountName, DeviceName
| order by count_ desc

### Defense Evasion

Clearing logs, disabling security tools, masquerading, and living-off-the-land evasion.

**Windows event log cleared (Sentinel: SecurityEvent 1102)**

In [ ]:
let lookback=30d;
SecurityEvent
| where TimeGenerated >= ago(lookback) and EventID == 1102
| project TimeGenerated, Computer, Account, Activity
| order by TimeGenerated desc

**Defender tampering indicators (Defender XDR: DeviceEvents)**

In [ ]:
let lookback=14d;
DeviceEvents
| where Timestamp >= ago(lookback)
| where ActionType has_any ("Tamper","Antivirus","Defender","ASR","Security")
| project Timestamp, DeviceName, ActionType, AdditionalFields, InitiatingProcessFileName, InitiatingProcessCommandLine
| order by Timestamp desc

**Suspicious use of 'wevtutil cl' (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=30d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where FileName =~ "wevtutil.exe"
| where ProcessCommandLine has " cl "
| project Timestamp, DeviceName, AccountName, ProcessCommandLine, InitiatingProcessFileName
| order by Timestamp desc

### Credential Access

LSASS access/dumps, Kerberoasting, credential theft tools, and suspicious authentication artifacts.

**Possible LSASS dumping via procdump/rundll32 (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=14d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where ProcessCommandLine has_any ("lsass", "comsvcs.dll", "MiniDump", "procdump", "rundll32")
| project Timestamp, DeviceName, AccountName, FileName, ProcessCommandLine, InitiatingProcessFileName, InitiatingProcessCommandLine
| order by Timestamp desc

**Kerberoasting candidate events (Sentinel: SecurityEvent 4769)**

In [ ]:
// Kerberos service ticket requests with RC4 (weak) encryption can be a signal (environment-dependent)
let lookback=7d;
SecurityEvent
| where TimeGenerated >= ago(lookback) and EventID == 4769
| extend TicketEncType=tostring(EventData.TicketEncryptionType), ServiceName=tostring(EventData.ServiceName)
| where TicketEncType in ("0x17","0x18") // RC4 variants commonly seen
| summarize count(), services=make_set(ServiceName, 10) by Account, Computer
| order by count_ desc

**Unusual credential-related commands (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=14d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where ProcessCommandLine has_any ("mimikatz","sekurlsa","lsadump","vault::list","cmdkey","rundll32 keymgr.dll")
| project Timestamp, DeviceName, AccountName, FileName, ProcessCommandLine, InitiatingProcessFileName
| order by Timestamp desc

### Discovery

Commands that enumerate users, groups, domains, shares, and system/network configuration.

**Built-in discovery commands (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=7d;
let cmds = dynamic(["whoami","ipconfig","nltest","net user","net group","net localgroup","dsquery","quser","qwinsta","systeminfo","tasklist"]);
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where ProcessCommandLine has_any (cmds)
| project Timestamp, DeviceName, AccountName, FileName, ProcessCommandLine
| order by Timestamp desc

**LDAP / AD enumeration via PowerShell modules (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=14d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where FileName in~ ("powershell.exe","pwsh.exe")
| where ProcessCommandLine has_any ("Get-ADUser","Get-ADComputer","Get-ADGroup","Get-Domain","Get-NetUser","Get-NetGroup")
| project Timestamp, DeviceName, AccountName, ProcessCommandLine
| order by Timestamp desc

### Lateral Movement

Remote logons, SMB/RDP connections, remote service creation, and PsExec-like behavior.

**Remote logon activity (Sentinel: SecurityEvent 4624 LogonType 3/10)**

In [ ]:
let lookback=7d;
SecurityEvent
| where TimeGenerated >= ago(lookback) and EventID == 4624
| extend LogonType = tostring(EventData.LogonType)
| where LogonType in ("3","10")  // 3=network, 10=remote interactive (RDP)
| project TimeGenerated, Computer, Account, IpAddress=tostring(EventData.IpAddress), LogonType, ProcessName=tostring(EventData.ProcessName)
| order by TimeGenerated desc

**Potential PsExec usage (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=14d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where ProcessCommandLine has_any ("psexec", "\\ADMIN$", "\\C$") or FileName in~ ("psexec.exe","paexec.exe")
| project Timestamp, DeviceName, AccountName, FileName, ProcessCommandLine, InitiatingProcessFileName
| order by Timestamp desc

**RDP/SMB outbound from endpoints (Defender XDR: DeviceNetworkEvents)**

In [ ]:
let lookback=7d;
DeviceNetworkEvents
| where Timestamp >= ago(lookback)
| where RemotePort in (3389, 445)
| summarize conns=count(), targets=dcount(RemoteIP), sampleTargets=make_set(RemoteIP, 10)
          by DeviceName, InitiatingProcessFileName, RemotePort
| order by conns desc

### Command and Control

Look for rare external destinations, suspicious DNS, and unusual ports/protocols.

**Rare outbound destinations per device (Defender XDR: DeviceNetworkEvents)**

In [ ]:
let lookback=14d;
let baseline=30d;
let base =
    DeviceNetworkEvents
    | where Timestamp between (ago(baseline) .. ago(lookback))
    | summarize knownRemotes=make_set(RemoteUrl, 1000) by DeviceName;
DeviceNetworkEvents
| where Timestamp >= ago(lookback)
| where isnotempty(RemoteUrl)
| summarize recent=make_set(RemoteUrl, 200) by DeviceName
| join kind=leftouter base on DeviceName
| extend new=set_difference(recent, knownRemotes)
| where array_length(new) > 0
| project DeviceName, new

**DNS query spikes (Sentinel: DnsEvents if available)**

In [ ]:
// If you ingest DNS logs into a DnsEvents table (connector-dependent)
let lookback=24h;
DnsEvents
| where TimeGenerated >= ago(lookback)
| summarize queries=count(), domains=dcount(Name) by ClientIP, bin(TimeGenerated, 1h)
| where queries > 500 and domains > 100
| order by queries desc

**Beacon-like periodic connections (Defender XDR: DeviceNetworkEvents)**

In [ ]:
let lookback=7d;
DeviceNetworkEvents
| where Timestamp >= ago(lookback)
| where RemoteIPType == "Public"
| summarize times=make_list(Timestamp, 200) by DeviceName, RemoteIP, RemotePort, InitiatingProcessFileName
| where array_length(times) > 20
| project DeviceName, RemoteIP, RemotePort, InitiatingProcessFileName, times

### Exfiltration

Large data transfers, uploads to cloud storage, and unusual egress.

**Large outbound network bytes (Defender XDR: DeviceNetworkEvents)**

In [ ]:
let lookback=7d;
DeviceNetworkEvents
| where Timestamp >= ago(lookback)
| where RemoteIPType == "Public"
| summarize totalBytes=sum(tolong(SentBytes)), conns=count(), urls=dcount(RemoteUrl)
          by DeviceName, InitiatingProcessFileName
| where totalBytes > 500000000  // 500MB
| order by totalBytes desc

**Potential cloud storage uploads (Defender XDR: DeviceNetworkEvents)**

In [ ]:
let lookback=7d;
let cloud = dynamic(["dropbox.com","drive.google.com","docs.google.com","box.com","mega.nz","onedrive.live.com"]);
DeviceNetworkEvents
| where Timestamp >= ago(lookback)
| where RemoteUrl has_any (cloud)
| summarize conns=count(), bytes=sum(tolong(SentBytes)) by DeviceName, RemoteUrl, InitiatingProcessFileName
| order by bytes desc

**Proxy/firewall exfil signals (Sentinel: CommonSecurityLog)**

In [ ]:
// Requires a firewall/proxy connector mapping to CommonSecurityLog
let lookback=7d;
CommonSecurityLog
| where TimeGenerated >= ago(lookback)
| where isnotempty(DestinationHostName) or isnotempty(DestinationIP)
| summarize bytesOut=sum(tolong(SentBytes)), sessions=count()
          by SourceIP, DestinationHostName, DestinationIP, ApplicationProtocol, bin(TimeGenerated, 1d)
| where bytesOut > 1000000000 // 1GB per day
| order by bytesOut desc

### Impact

Ransomware-style patterns, destructive commands, shadow-copy deletion, mass file changes.

**Shadow copy deletion (Defender XDR: DeviceProcessEvents)**

In [ ]:
let lookback=30d;
DeviceProcessEvents
| where Timestamp >= ago(lookback)
| where ProcessCommandLine has_any ("vssadmin delete shadows","wmic shadowcopy delete","bcdedit /set", "wevtutil cl")
| project Timestamp, DeviceName, AccountName, FileName, ProcessCommandLine, InitiatingProcessFileName
| order by Timestamp desc

**Mass file modifications with suspicious extensions (Defender XDR: DeviceFileEvents)**

In [ ]:
let lookback=7d;
let suspiciousExt = dynamic([".locked",".encrypt",".crypted",".ryk",".conti",".blackcat",".lockbit",".clop"]);
DeviceFileEvents
| where Timestamp >= ago(lookback)
| where FileName has_any (suspiciousExt)
| summarize files=count(), sample=make_set(FileName, 10) by DeviceName, InitiatingProcessFileName, bin(Timestamp, 1h)
| where files >= 50
| order by files desc

**Ransom note filenames (Defender XDR: DeviceFileEvents)**

In [ ]:
let lookback=30d;
let notes = dynamic(["README","RECOVER","DECRYPT","HOW_TO","HELP","RESTORE"]);
DeviceFileEvents
| where Timestamp >= ago(lookback)
| where FileName has_any (notes) and FileName has_any (".txt",".html",".hta")
| summarize count(), sample=make_set(FileName, 20) by DeviceName, InitiatingProcessFileName
| order by count_ desc

---

## Next steps

1. Run the **catalog downloader** to get every query file currently in the public repos.  
2. Use `search_titles()` to find content by technique/tool/campaign name (e.g., `mimikatz`, `Kerberoast`, `psexec`, `rundll32`).  
3. Use `load_query(repo, rel_path)` to open the exact KQL text for a query you want to run.  
4. Export the index to CSV/JSON for your own reporting and tracking.

If you want, you can extend the tactic inference to also extract **technique IDs** (e.g., `T1059`) where repositories include them in folder names or file headers.
